In [1]:
%cd ..

/home/runner/work/clima-perf/clima-perf


In [2]:
import glob
import os
from datetime import datetime
import json

import polars as pl
from plotly.graph_objects import Figure

logs = glob.glob("runs/amip/*/output.log")

dates = []
sypds = []
run_locs = []

for log in logs:
    date = datetime.fromisoformat(log.split("/")[-2])
    dates.append(date)
    run_locs.append("calkit")
    with open(log, "r") as f:
        lines = f.readlines()
    sypd = None
    found_step_40 = False
    for line in lines:
        if (
            "n_steps_completed = 40" in line
            or "n_steps_completed = 41" in line
        ):
            found_step_40 = True
        if found_step_40 and "estimated_sypd = " in line:
            sypd = float(
                line.split("estimated_sypd = ")[1].split()[0].replace('"', "")
            )
            break
    sypds.append(sypd)

# Now read any fetched buildkite logs
bk_jobs = glob.glob("buildkite/*.json")
for job_fpath in bk_jobs:
    with open(job_fpath) as f:
        job = json.load(f)
    date = datetime.fromisoformat(job["created_at"])
    dates.append(date)
    run_locs.append("buildkite")
    sypd = None
    found_step_40 = False
    for line in job["raw_log_txt"].split("\n"):
        if (
            "n_steps_completed = 40" in line
            or "n_steps_completed = 41" in line
        ):
            found_step_40 = True
        if found_step_40 and "estimated_sypd = " in line:
            sypd = float(
                line.split("estimated_sypd = ")[1].split()[0].replace('"', "")
            )
            break
    sypds.append(sypd)

df = pl.DataFrame({"date": dates, "sypd": sypds, "run_from": run_locs})
df = df.sort("date")

# Add normalized SYPD column; normalize by the first non-null SYPD value
first_sypd = df["sypd"].drop_nulls()[0]
df = df.with_columns((pl.col("sypd") / first_sypd).alias("speedup"))

print(df)

os.makedirs("results", exist_ok=True)
df.write_csv("results/amip.csv")

fig: Figure = df.drop_nulls().to_pandas().plot(
    x="date",
    y="speedup",
    title="Flagship AMIP benchmark (16 horiz. elem.)",
    backend="plotly",
)  # type: ignore

# Use lines and markers
fig.update_traces(mode="lines+markers")
fig.update_layout(yaxis_title="Speedup", xaxis_title="Date")

# Save as PNG and JSON
os.makedirs("figures", exist_ok=True)
fig.write_image("figures/amip-speedup.png")
fig.write_json("figures/amip-speedup.json")
fig

shape: (42, 4)
┌─────────────────────────┬───────┬───────────┬──────────┐
│ date                    ┆ sypd  ┆ run_from  ┆ speedup  │
│ ---                     ┆ ---   ┆ ---       ┆ ---      │
│ datetime[μs]            ┆ f64   ┆ str       ┆ f64      │
╞═════════════════════════╪═══════╪═══════════╪══════════╡
│ 2025-10-15 00:00:00     ┆ null  ┆ calkit    ┆ null     │
│ 2025-11-01 00:00:00     ┆ 0.059 ┆ calkit    ┆ 1.0      │
│ 2025-11-02 00:00:00     ┆ 0.06  ┆ calkit    ┆ 1.016949 │
│ 2025-11-03 00:00:00     ┆ 0.048 ┆ calkit    ┆ 0.813559 │
│ 2025-11-04 00:00:00     ┆ 0.06  ┆ calkit    ┆ 1.016949 │
│ …                       ┆ …     ┆ …         ┆ …        │
│ 2026-02-17 02:01:08.722 ┆ 0.115 ┆ buildkite ┆ 1.949153 │
│ 2026-02-18 02:01:12.455 ┆ 0.115 ┆ buildkite ┆ 1.949153 │
│ 2026-02-19 02:01:10.265 ┆ 0.103 ┆ buildkite ┆ 1.745763 │
│ 2026-02-20 02:01:10.023 ┆ 0.121 ┆ buildkite ┆ 2.050847 │
│ 2026-02-24 02:01:09.226 ┆ 0.121 ┆ buildkite ┆ 2.050847 │
└─────────────────────────┴───────┴──────